In [ ]:
# 1. Setting up Keras Tuner
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

(x_train,y_train),(x_test,y_test) = mmist.load_data()
x_val, y_val  = x_val/255.0 , y_val/255.0

print(f'Shape of x_train: {x_train.shape}')
print(f'Shape of x_test: {x_val.shape}')

# 2. build model

def build_model(hp):
    model = Sequential([
    model.add(Flatten(input_shape=(28,28)),
              Dense(units = hp.Int("unit",min_value = 32, max_value = 512, step = 32),
                    activation = 'relu'),
              Dense(10, activation = "softmax"))
    ])

    model.compile(
        qtimizer = Adam(learning_rate =hp.Float('learning_rate,'min_val`=1e-4, max_value = 1e-2, sampling = "log")),
                  loss = "sparse_categorical_crossentropy",
        metrics = ["accuracy"]
    )

    return model

#3.Configuring the hyperparameter search objective

tuner = kt.RandomSearch(
    build_model,
    objective = "val_accuracy",#val_accuracy 를 최적하는것이 objective
    max_trials = 10, # 최대 시도 횟수
    executions_per_trial = 2, # 2번 실행해서 평균결과를 사용
    directory = "my_dir",
    project_name = "mnist_to_keras tuner"
)

tuner.search_space_summary()


# 4. Running the hyperparameter search
tuner.search(x_train, y_train, epochs =5, validation_data = (x_val, y_val))
tuner.results_summary()

# 5. Analyzing and using the best hyperparameter
best_hy = tuner.get_best_hyperparameters(num_trials = 1)[0]
# (num_trials = 1) 가장 성능이 좋았던 "1"개의 파라미터 조합을 가져옴
# [0] 그중에서 첫번쨰 값을 불러오기
print(f"""

The optimal number of units in the first dense layer is {best_hps.get('units')}.

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.

""")


# Rebuild and train the model with best hyperparameters
model = tuner.hypermodel.build(best_hy)
history = model.fit(x_train, y_train, epochs = 10, validation_split= 0.2)

#evaluate the model on the validation set
val_loss, val_acc = model.evaluate(x_val, y_val)
print(f"Validation accuracy: {val_acc}")



